<a href="https://colab.research.google.com/github/dongxiat/MyColabHub/blob/main/DepthFlow_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install ModernGL to render with the Tesla GPU in Colab
import os
import subprocess
import json

# --- Bước 1: Cài đặt các thư viện cần thiết (nếu chưa có) ---
print("⏳ Đang cài đặt thư viện hệ thống và moderngl...")
# Chạy ở chế độ yên lặng để không làm nhiễu output
!apt-get install -y libegl1-mesa-dev libgles2-mesa-dev > /dev/null
!pip install moderngl[headless] > /dev/null
print("✅ Cài đặt hoàn tất!")


# --- Bước 2: Tìm kiếm và cấu hình Driver NVIDIA cho EGL ---
print("\n⚙️ Đang tìm kiếm thư viện EGL của NVIDIA...")

# Tìm đường dẫn đến thư viện libEGL_nvidia.so.0
# Đây là file driver EGL thực sự của NVIDIA
find_cmd = 'find / -name "libEGL_nvidia.so.0" 2>/dev/null'
result = subprocess.run(find_cmd, shell=True, capture_output=True, text=True)
nvidia_egl_path = result.stdout.strip().split('\n')[0]

if nvidia_egl_path:
    print(f"  -> Đã tìm thấy driver EGL của NVIDIA tại: {nvidia_egl_path}")

    # EGL cần một file JSON để biết vị trí của driver.
    # Nếu file này không có sẵn, chúng ta sẽ TẠO RA NÓ.
    vendor_json_path = "/usr/share/glvnd/egl_vendor.d/10_nvidia.json"

    # Tạo thư mục nếu nó chưa tồn tại
    os.makedirs(os.path.dirname(vendor_json_path), exist_ok=True)

    # Tạo nội dung cho file JSON
    json_content = {
        "file_format_version": "1.0.0",
        "ICD": {
            "library_path": nvidia_egl_path
        }
    }

    # Ghi file JSON
    with open(vendor_json_path, 'w') as f:
        json.dump(json_content, f, indent=4)
    print(f"  -> Đã tạo file cấu hình EGL tại: {vendor_json_path}")

    # Một số môi trường cần cả LD_LIBRARY_PATH để tìm các thư viện phụ thuộc
    nvidia_lib_dir = os.path.dirname(nvidia_egl_path)
    if 'LD_LIBRARY_PATH' in os.environ:
        os.environ['LD_LIBRARY_PATH'] = f"{nvidia_lib_dir}:{os.environ['LD_LIBRARY_PATH']}"
    else:
        os.environ['LD_LIBRARY_PATH'] = nvidia_lib_dir
    print(f"  -> Đã cập nhật LD_LIBRARY_PATH.")

else:
    print("❌ LỖI: Không thể tìm thấy 'libEGL_nvidia.so.0'. Không thể cấu hình GPU.")
    # Dừng ở đây nếu không tìm thấy driver
    exit()

# --- Bước 3: Khởi tạo ModernGL và kiểm tra ---
# Phải import moderngl SAU KHI đã thiết lập biến môi trường
import moderngl

print("\n⏳ Đang tạo context ModernGL với backend 'egl'...")
try:
    ctx = moderngl.create_standalone_context(backend='egl')

    print("\n--- KẾT QUẢ ---")
    gl_vendor = ctx.info.get('GL_VENDOR', '')
    gl_renderer = ctx.info.get('GL_RENDERER', '')

    print(f"GL_VENDOR: {gl_vendor}")
    print(f"GL_RENDERER: {gl_renderer}")

    if 'nvidia' in gl_vendor.lower() or 'nvidia' in gl_renderer.lower():
        print("\n🚀 Tuyệt vời! ModernGL đã kết nối thành công với GPU NVIDIA.")
    else:
        print(f"\n❌ Thất bại! ModernGL vẫn đang sử dụng trình render phần mềm: '{gl_renderer}'.")
        print("💡 Gợi ý: Vấn đề có thể do môi trường Colab này quá khác biệt. Thử 'Factory reset runtime' và chạy lại.")

except Exception as e:
    print(f"\n❌ LỖI nghiêm trọng khi tạo context ModernGL: {e}")

print(ctx.info)


⏳ Đang cài đặt thư viện hệ thống và moderngl...
✅ Cài đặt hoàn tất!

⚙️ Đang tìm kiếm thư viện EGL của NVIDIA...
  -> Đã tìm thấy driver EGL của NVIDIA tại: /usr/lib64-nvidia/libEGL_nvidia.so.0
  -> Đã tạo file cấu hình EGL tại: /usr/share/glvnd/egl_vendor.d/10_nvidia.json
  -> Đã cập nhật LD_LIBRARY_PATH.

⏳ Đang tạo context ModernGL với backend 'egl'...

--- KẾT QUẢ ---
GL_VENDOR: NVIDIA Corporation
GL_RENDERER: Tesla T4/PCIe/SSE2

🚀 Tuyệt vời! ModernGL đã kết nối thành công với GPU NVIDIA.
{'GL_VENDOR': 'NVIDIA Corporation', 'GL_RENDERER': 'Tesla T4/PCIe/SSE2', 'GL_VERSION': '3.3.0 NVIDIA 550.54.15', 'GL_POINT_SIZE_RANGE': (1.0, 2047.0), 'GL_SMOOTH_LINE_WIDTH_RANGE': (0.5, 10.0), 'GL_ALIASED_LINE_WIDTH_RANGE': (1.0, 10.0), 'GL_POINT_FADE_THRESHOLD_SIZE': 1.0, 'GL_POINT_SIZE_GRANULARITY': 0.125, 'GL_SMOOTH_LINE_WIDTH_GRANULARITY': 0.125, 'GL_MIN_PROGRAM_TEXEL_OFFSET': -8.0, 'GL_MAX_PROGRAM_TEXEL_OFFSET': 7.0, 'GL_MINOR_VERSION': 3, 'GL_MAJOR_VERSION': 3, 'GL_SAMPLE_BUFFERS': 0, 'GL_S

In [ ]:
#@title Run Depthflow
%cd /content/
!pip install git+https://github.com/BrokenSource/DepthFlow.git -q
!depthflow gradio --share

/content
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 97.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 17.6 MB/